In [ ]:
!nvidia-smi

Sat May 17 13:15:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/minhnhatvt/label-distribution-learning-fer-tf.git
%cd label-distribution-learning-fer-tf

Cloning into 'label-distribution-learning-fer-tf'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 65 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 27.89 KiB | 9.30 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/label-distribution-learning-fer-tf


In [ ]:
!chmod +x install.sh
!./install.sh

./install.sh: line 1: conda: command not found
  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-l2p4j5w1
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/classification_models.git /tmp/pip-req-build-l2p4j5w1
  Resolved https://github.com/qubvel/classification_models.git to commit a0f006e05485a34ccf871c421279864b0ccd220b
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.7 MB/s eta 0:00:00
  Created wheel for image_classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20031 sha256=aa008dbb9459e42c080eb43848548dfbf725fa343a8a1f7efc99d4a8a1e535a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-97yvnf1j/wheels/14/43/ed/2d68b197c5260e7fe6b7d0fab20b997b6d365f6f5732316952
Successfully built image_classifiers


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving resnet101.h5 to resnet101.h5


In [ ]:
import os
os.makedirs("data/rafdb", exist_ok=True)
!unzip -q aligned.zip -d data/rafdb/
shutil.move("raf_test.csv", "data/rafdb/raf_test.csv")
shutil.move("raf_train.csv", "data/rafdb/raf_train.csv")

'data/rafdb/raf_train.csv'

In [ ]:
import shutil
os.makedirs("pretrained", exist_ok=True)
shutil.move("resnet101.h5", "pretrained/resnet101.h5")
shutil.move("resnet18.h5", "pretrained/resnet18.h5")
shutil.move("resnet50.h5", "pretrained/resnet50.h5")

'pretrained/resnet50.h5'

In [ ]:
!sed -i 's/\blr\b/learning_rate/g' src/data_utils.py src/eval.py src/models.py src/train.py src/utils.py
!sed -i 's/config\.learning_rate/config.lr/g' src/data_utils.py src/eval.py src/models.py src/train.py src/utils.py
!sed -i "s/SGD(config.lamb_lr, config.lamb_beta)/SGD(float(config.lamb_lr), config.lamb_beta)/g" src/train.py
!sed -i "s/SGD(float(config.lamb_lr), config.lamb_beta)/SGD(float(config.lamb_lr), float(config.lamb_beta))/g" src/train.py
!find src/ -type f -name "*.py" -exec sed -i 's/\.reset_states()/\.reset_state()/g' {} +
!sed -i 's/float(optimizer.learning_rate(optimizer.iterations))/float(optimizer.learning_rate)/g' src/train.py
!find src/ -type f -name "*.py" -exec sed -i "s|/Model|/model.weights.h5|g" {} +
!find src/ -type f -name "*.py" -exec sed -i "/model.save_weights/s|model.save_weights(f'\(.*\)/model.weights.h5')|save_dir = f'\1'; tf.io.gfile.makedirs(save_dir); model.save_weights(f'{save_dir}/model.weights.h5')|g" {} +


In [ ]:
!python src/train.py \
  --cfg=config_resnet50_raf \
  --train_data_path=data/rafdb/raf_train.csv \
  --train_image_dir=data/rafdb/aligned

2025-05-17 20:43:58.633812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747514638.653865  118228 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747514638.659905  118228 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/content/label-distribution-learning-fer-tf/src/data_utils.py:66: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'black':
/content/label-distribution-learning-fer-tf/src/data_utils.py:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is 'white':
/content/label-distribution-learning-fer-tf/src/data_utils.py:70: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is 'random

In [ ]:
# only run for resume
!python src/train.py --cfg=config_resnet50_raf --train_data_path=data/rafdb/raf_train.csv --train_image_dir=data/rafdb/aligned

In [ ]:
!sed -i "s|model.load_weights(.*)|tf.train.Checkpoint(model=model).restore(tf.train.latest_checkpoint('checkpoints/')).expect_partial()|g" /content/label-distribution-learning-fer-tf/src/eval.py
!python src/eval.py --cfg=config_resnet50_raf --trained_weights=trained_weights/trained_resnet50_raf --test_data_path=data/rafdb/raf_test.csv --test_image_dir=data/rafdb/aligned

2025-05-17 22:44:03.090922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747521843.110604  148435 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747521843.116657  148435 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
{'name': 'Base Config', 'backbone': 'resnet50', 'feature_dim': 512, 'pretrained': 'msceleb', 'input_size': [112, 112], 'pad_size': 4, 'batch_size': 16, 'num_parallel_calls': -1, 'optimizer': 'adam', 'lr': 0.0001, 'lr_decay': 0.1, 'lr_steps': [10, 30], 'gamma': 0.01, 'num_neighbors': 4, 'lamb_init': 0.5, 'lamb_lr': 10, 'lamb_beta': 0, 'num_classes': 7, 'class_names': ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger'],

In [ ]:
%cd ..
!zip -r label-distribution-learning-fer-tf.zip label-distribution-learning-fer-tf
from google.colab import files
files.download("label-distribution-learning-fer-tf.zip")

Streaming output truncated to the last 5000 lines.
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/test_1076_aligned.jpg (deflated 6%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_01819_aligned.jpg (deflated 6%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_07408_aligned.jpg (deflated 5%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_10713_aligned.jpg (deflated 6%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_03968_aligned.jpg (deflated 5%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_08404_aligned.jpg (deflated 6%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_08873_aligned.jpg (deflated 6%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/train_04202_aligned.jpg (deflated 4%)
  adding: label-distribution-learning-fer-tf/data/rafdb/aligned/test_3060_aligned.jpg (deflated 5%)
  adding: label-distribution-learni

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install scikit-learn matplotlib seaborn

In [ ]:
import sys
sys.path.append("cfg_files")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE

from src.models import create_model
from src.data_utils import get_test_dataset

import config_resnet50_raf
config = config_resnet50_raf.config


In [ ]:
def cross_entropy_loss(p, q):
    return -tf.reduce_sum(p * tf.math.log(q + 1e-7), axis=-1)

def kl_divergence(p, q):
    return tf.reduce_sum(p * tf.math.log((p + 1e-7) / (q + 1e-7)), axis=-1)


In [1]:
def evaluate_model(model, test_dataset, class_names):
    y_true, y_pred_top1 = [], []
    ce_losses, kl_losses = [], []
    features_list, predicted_distributions = [], []

    for x_batch, _, emotion_true in test_dataset:
        features, pred_dist = model(x_batch, training=False)
        true_dist = tf.one_hot(emotion_true, depth=len(class_names))

        predicted_distributions.append(pred_dist.numpy())
        features_list.append(features.numpy())
        y_true.extend(emotion_true.numpy())
        y_pred_top1.extend(tf.argmax(pred_dist, axis=-1).numpy())

        ce_losses.extend(cross_entropy_loss(true_dist, pred_dist).numpy())
        kl_losses.extend(kl_divergence(true_dist, pred_dist).numpy())

    y_true = np.array(y_true)
    y_pred_top1 = np.array(y_pred_top1)
    predicted_distributions = np.vstack(predicted_distributions)
    features_embedded = np.vstack(features_list)

    print("Top-1 Accuracy:", np.mean(y_pred_top1 == y_true))
    print("Mean Cross-Entropy Loss:", np.mean(ce_losses))
    print("Mean KL Divergence:", np.mean(kl_losses))

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_top1, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred_top1)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

    tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=0)
    reduced = tsne.fit_transform(features_embedded)
    plt.figure(figsize=(10, 8))
    for i, label in enumerate(class_names):
        idx = y_true == i
        plt.scatter(reduced[idx, 0], reduced[idx, 1], label=label, alpha=0.6)
    plt.legend()
    plt.title("t-SNE of Emotion Embeddings")
    plt.show()


In [2]:

test_csv = "data/rafdb/raf_test.csv"
image_dir = "data/rafdb/aligned"
test_dataset = get_test_dataset(test_csv, image_dir, config)

model = create_model(config)
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint("checkpoints/")).expect_partial()
print(" Model checkpoint loaded.")

evaluate_model(model, test_dataset, class_names=config.class_names)


NameError: name 'get_test_dataset' is not defined